# Compare_MOST_GeoClaw.ipynb


In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import CHTtools
from importlib import reload
reload(CHTtools)
from clawpack.visclaw import geoplot, colormaps

In [ ]:
fname_nc = 'MOST_seaside_tsunami_data.nc'  # for buried-random-mur13-deep
mostResults = CHTtools.TsunamiModelResults(ncfile=fname_nc,format='most')
mostResults.load_results()

In [ ]:
mostResults.make_all_qoi_fcns()

In [ ]:
event = 'buried-random-mur13-deep'
fgno = 3
fname_nc = fname_nc = '%s-fgout%s.nc' % (event,str(fgno).zfill(3))
print('Loading results from %s' % fname_nc)
geoclawResults = CHTtools.TsunamiModelResults(ncfile=fname_nc, format='geoclaw')
geoclawResults.load_results()

In [ ]:
geoclawResults.make_all_qoi_fcns()

In [ ]:
def extract_grid_onshore(modelResults, t):
    """
    extract depth and speed at time tt (minutes) at onshore points (nan offshore)
    if make_plots:  also plot depth on contour plot of topo
    """
    depth_t = modelResults.depth_fcn((modelResults.X,modelResults.Y,t))
    depth_t = where(isnan(depth_t), 0, depth_t)
    depth_onshore = where(modelResults.B0>0, depth_t, nan)
    u_t = modelResults.u_fcn((modelResults.X,modelResults.Y,t))
    v_t = modelResults.v_fcn((modelResults.X,modelResults.Y,t))
    speed_t = sqrt(u_t**2 + v_t**2)
    speed_t = where(isnan(speed_t), 0, speed_t)
    speed_onshore = where(modelResults.B0>0, speed_t, nan)
    return depth_onshore, speed_onshore

In [ ]:
def compare_depth(t):
    geoclaw_depth_onshore, geoclaw_speed_onshore = extract_grid_onshore(geoclawResults, t)
    most_depth_onshore, most_speed_onshore = extract_grid_onshore(mostResults, t)

    cmap = plt.get_cmap('YlOrRd')
    cmap.set_under([.1,.7,.1])
    cmap.set_over('m')
    
    fig,axs = subplots(1,2,figsize=(12,8))
    ax = axs[0]
    pc = ax.pcolormesh(mostResults.X, mostResults.Y, most_depth_onshore, cmap=cmap)
    ax.set_aspect(1/cos(pi*46/180))
    pc.set_clim(0.05,6)
    colorbar(pc, extend='max',label='meters',shrink=0.8)
    #ax.ticklabel_format(useOffset=False)
    #ax.set_xticks(ax.get_xticks(),rotation=20)
    ax.axis(geoclawResults._extent)
    ax.set_title('MOST \n Onshore depth at t = %.2f minutes' % (t/60), 
                 fontsize=10);
    
    ax = axs[1]
    pc = ax.pcolormesh(geoclawResults.X, geoclawResults.Y, geoclaw_depth_onshore, cmap=cmap)
    ax.set_aspect(1/cos(pi*46/180))
    pc.set_clim(0.05,6)
    colorbar(pc, extend='max',label='meters',shrink=0.8)
    #ax.ticklabel_format(useOffset=False)
    #ax.set_xticks(ax.get_xticks(),rotation=20)
    ax.axis(geoclawResults._extent)
    ax.set_title('GeoClaw \n Onshore depth at t = %.2f minutes' % (t/60), 
                 fontsize=10);

In [ ]:
compare_depth(35*60)

In [ ]:
compare_depth(45*60)

In [ ]:
compare_depth(55*60)

## Compare some transects

In [ ]:
def compare_transect(y0,t):
    figure(figsize=(10,4))
    
    x = mostResults.x
    most_eta_y0 = mostResults.eta_fcn((x,y0,t))
    plot(x, most_eta_y0, color='m', label='MOST surface')
    
    x = geoclawResults.x
    geoclaw_eta_y0 = geoclawResults.eta_fcn((x,y0,t))
    plot(x, geoclaw_eta_y0, color='b', label='GeoClaw surface')

    B_y0 = geoclawResults.B0_fcn((x,y0))
    plot(x, B_y0, 'g', label='topo')
    xlim(x[0],x[-1])
    ylim(-10,10)
    ticklabel_format(useOffset=False)
    grid(True)
    legend(loc='lower right', framealpha=1)
    title('Transect at y = %.5f, t = %.2f minutes' % (y0,t/60));

In [ ]:
compare_transect(y0 = 46.01, t = 35*60.)

In [ ]:
compare_transect(y0 = 46.01, t = 45*60.)

In [ ]:
compare_transect(y0 = 46.01, t = 55*60.)

## Compare some time series:

In [ ]:
def compare_gauge(xg,yg):
    figure(figsize=(10,4))

    tg = mostResults.t
    depth_tg = mostResults.depth_fcn((xg,yg,tg))
    eta_tg = mostResults.eta_fcn((xg,yg,tg))
    Bg = mostResults.B0_fcn((xg,yg))
    plot(tg, eta_tg, 'm', label='MOST surface')
    plot(tg, depth_tg, 'r', label='MOST depth')

    tg = geoclawResults.t
    depth_tg = geoclawResults.depth_fcn((xg,yg,tg))
    eta_tg = geoclawResults.eta_fcn((xg,yg,tg))
    Bg = geoclawResults.B0_fcn((xg,yg))
    plot(tg, eta_tg, 'b', label='GeoClaw surface')
    plot(tg, depth_tg, 'c', label='GeoClaw depth')
    
    grid(True)
    legend(loc='upper left', framealpha=1)
    title('%s -- extracted time series at \nxg = %.5f, yg = %.5f, B = %.2f' % (event,xg,yg,Bg));

### At an offshore point:

In [ ]:
compare_gauge(xg = -123.9500, yg = 46.0100)

### At an onshore point:

In [ ]:
compare_gauge(xg = -123.9300, yg = 46.0000)